In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

## filepath

In [2]:
#train_data = "../Switchboard-Corpus/swda_data/train_set.txt"
#test_data = "../Switchboard-Corpus/swda_data/test_set.txt"
#val_data = "../Switchboard-Corpus/swda_data/val_set.txt"

## 練習

In [3]:
swda_pass = "../Switchboard-Corpus/swda_data/"
dir_list = ["train", "test", "val"]

In [6]:
train_dataset = []
f = open(swda_pass + "train/2005.txt", "r", encoding='utf-8')
for row in f:
    train_dataset.append(row.strip())
f.close()

In [7]:
utter_user = []
utter = []
utter_label = []

for j in train_dataset:
    for i,v in enumerate(j.split("|")):
        if i == 0:
            utter_user.append(v)
        elif i==1:
            utter.append(v)
        else:
            utter_label.append(v)
print("finish")

finish


In [11]:
labels = set(utter_label)
utter_labels = []
for i in utter_label:
    for j,v in enumerate(labels):
        if i == v:
            utter_labels.append(j)
        

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [12]:
train_data = tf.data.Dataset.from_tensor_slices((utter, utter_labels))

In [13]:
train_data

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.int32)>

In [7]:
train_dataset = []
f = open(train_data, "r", encoding='utf-8')
for row in f:
    train_dataset.append(row.strip())
f.close()

In [8]:
len(train_dataset)

192390

In [9]:
test_dataset = []
f = open(test_data, "r", encoding='utf-8')
for row in f:
    test_dataset.append(row.strip())
f.close()

In [10]:
len(test_dataset)

4078

In [11]:
val_dataset = []
f = open(val_data, "r", encoding='utf-8')
for row in f:
    val_dataset.append(row.strip())
f.close()

In [12]:
len(val_dataset)

3272

## データの分割

In [13]:
utter_user = []
utter = []
utter_label = []

In [14]:
train_dataset[0]
train_dataset[0].split("|")

['A', 'Okay.', 'fo_o_fw_"_by_bc']

In [15]:
for j in train_dataset:
    for i,v in enumerate(j.split("|")):
        if i == 0:
            utter_user.append(v)
        elif i==1:
            utter.append(v)
        else:
            utter_label.append(v)
print("finish")

finish


## ウィンドウサイズ

In [16]:
window_size = 5

In [17]:
train_datasets = []
lists = []
for i in range(len(train_dataset)):
    lists.append(train_dataset[i])
    if i%window_size == 4:
        train_datasets.append(lists)
        lists = []

In [18]:
len(train_datasets)

38478

## Tensorflow

In [34]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [42]:
utter_train_dataset = tf.data.Dataset.from_tensor_slices((utter, utter_label))

In [57]:
vocabulary_set = set()
for text_tensor, _ in utter_train_dataset:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)

21852

In [58]:
tokenizer = tfds.features.text.Tokenizer()
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set, tokenizer=tokenizer)

In [59]:
encoder.vocab_size

21854

In [62]:
def encode(token, label):
    token = encoder.encode(token.numpy())
    label = np.array([label])
    return token, label

def tf_encoder(utter, label):
    return tf.py_function(encode, [utter, label],[tf.int64, tf.int64])

MAX_LENGTH = 40
def filter_max_length(x,y,max_length=MAX_LENGTH):
    return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

In [63]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [64]:
train_data_set = utter_train_dataset.map(tf_encoder)
train_data_set = train_data_set.filter(filter_max_length)
train_data_set = train_data_set.cache()
train_data_set = train_data_set.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([-1], [-1]))
train_data_set.prefetch(tf.data.experimental.AUTOTUNE)

<PrefetchDataset shapes: ((None, None), (None, None)), types: (tf.int64, tf.int64)>

In [68]:
def CreateModel(vocab_size, BATCH_SIZE):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, 300, batch_input_shape=[BATCH_SIZE, None]),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')),
        tf.keras.layers.Dense(vocab_size)
    ])

In [ ]:
model = CreateModel(vocab_size, BATCH_SIZE)

In [5]:
utter_embedding = tf.keras.layers.Embedding(1000, 300)

In [19]:
token = tf.keras.preprocessing.text.Tokenizer()

In [20]:
token.fit_on_texts(utter)
print(token.document_count)

192390


### ベクトル化

In [21]:
sequence_data = token.texts_to_sequences(utter)

In [21]:
## バイナリ表現
matrix_binary = token.sequences_to_matrix(sequence_data, "binary")

In [1]:
## カウント表現
matrix_count = token.sequences_to_matrix(sequence_data, "count")

NameError: name 'token' is not defined

In [22]:
## tfidf
matrix_tfidf = token.sequences_to_matrix(sequence_data, "tfidf")